In [5]:
#############################
#          Imports          #
#############################

import pandas as pd
import numpy as np
import requests
import json
import datetime
import time
import os

#############################

#setting current directory as object for later purposes
currentdir = os.getcwd()

#API Authentication
USER = 'xxxxxxxx'
PASS = 'xxxxxxxx'
t = requests.Session()
t.auth = (USER, PASS)

In [6]:
#Retrieve IDs from csv
os.chdir('R:\\7-Job Data')
#file name has to be updated whenever script runs, to account for newest data
file = 'Cleansed Data.xlsx'
empIDS = pd.read_excel(file)
empIDS = empIDS['Empl Id']
#empIDS.keys()# Treat this as a list. You can loop through the ids with indexes. Ex: empIDS[0]

In [7]:

########################### 
#      Load API Data      #
###########################

final = []
#First Loop through all the ids 
for i in empIDS:
    #when python reads the data from a csv, 
    #it removes the leading 0 in a number/id so you have to put it back in
    url = "https://apis.berkeley.edu/uat/hcm/v2/employees/"+ str(0) + str(i) +"/demographics"
    try:
        demog = json.loads(t.get(url).text)
    except ValueError:
        continue
    data = demog["PersonResponse"]["PersonList"]["Person"][0]
    EmplID = data['EmplID']
    EmplType = data['EmplType']
    EffectiveDate = data["EffectiveDate"]
    EffectiveStatus = data["EffectiveStatus"]
    EffectiveStatusDesc = data["EffectiveStatusDescr"]
    if "BusinessTitle" not in data.keys():
        BusinessTitle = ''
    else:
        BusinessTitle = data["BusinessTitle"]
    if "HomeDept" not in data.keys():
        HomeDept = ''
    else:
        HomeDept = data["HomeDept"]
    #Identifiers is nested dictionary format where values are keys and values (HCM-EMPLOYEE-ID is key, for example)
    HCMEmplID = data["Identifiers"]["Identifier"][0]['id']
    CSStudentID = data["Identifiers"]["Identifier"][1]['id']
    UID = data["Identifiers"]["Identifier"][2]['id']
    #Ethnicity
    EthnicityGroupCode = data["Ethnicities"]["Ethnicity"]["Group"]["code"]
    EthnicityGroupDesc = data["Ethnicities"]["Ethnicity"]["Group"]["description"]
    #Gender
    GenderGroupCode = data["Gender"]["genderIdentity"]["code"]
    GenderGroupDesc = data["Gender"]["genderIdentity"]["description"]
    #Citizenship
    if "USACountry" not in data.keys():
        CitizenshipCode, CitizenshipDesc, MilitaryCode, MilitaryDesc = '', '', '', ''
    else:
        CitizenshipCode = data["USACountry"]["CitizenshipStatus"]["code"]
        CitizenshipDesc = data["USACountry"]["CitizenshipStatus"]["description"]
        #Military Status
        MilitaryCode = data["USACountry"]["MilitaryStatus"]["code"]
        MilitaryDesc = data["USACountry"]["MilitaryStatus"]["description"]
    #Names
    NameCode = data["Names"]["Name"][0]["type"]["code"]
    NameFamily = data["Names"]["Name"][0]["familyName"]
    NameGiven = data["Names"]["Name"][0]["givenName"]
    NameEffectiveDate = data["Names"]["Name"][0]["effectiveDate"] #Might be birthday, need confirmation
    #Creating a dictionary for Person
    result = {"EmplID" : EmplID, "EmplType" : EmplType, "EffectiveDate": EffectiveDate, "EffectiveStatus": EffectiveStatus,
              "EffectiveStatusDesc": EffectiveStatusDesc, "BusinessTitle": BusinessTitle, "HomeDept": HomeDept,
              "HCMEmplID": HCMEmplID, "CSStudentID": CSStudentID, "UID": UID, "EthnicityGroupCode": EthnicityGroupCode, 
              "EthnicityGroupDesc": EthnicityGroupDesc, "GenderGroupCode": GenderGroupCode, "GenderGroupDesc": GenderGroupDesc, 
              "CitizenshipCode": CitizenshipCode, "CitizenshipDesc": CitizenshipDesc,
              "MilitaryCode": MilitaryCode, "MiliaryDesc": MilitaryDesc, "NameCode": NameCode, "FamilyName": NameFamily,
              "GivenName": NameGiven, "NameEffectiveDate": NameEffectiveDate
             }
    #Appending individual data to whole data
    final.append(result)
#create a Pandas DataFrame with the list of dictionaries 
#converting to data frame
df = pd.DataFrame(final)
#saves in the O drive - Data Analytics- Python Script Folder
os.chdir('R:\\4-EmployeeAPI')
#creating object for today's date and adding it to filename
filename = "Employee_Demographics_SA" + time.strftime("%Y%m%d") + ".csv"
#saving as a csv file
df.to_csv(filename)